In [1]:
from sklearn.metrics import (accuracy_score, 
                            confusion_matrix, 
                            classification_report,
                            plot_confusion_matrix)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier

In [2]:
data = pd.read_csv('./data/mlb-pitches.csv')
data.drop(columns = ['Unnamed: 0'], inplace = True) 

pd.set_option('max_columns', None)
print(data.shape)
data.head(3)

(708653, 76)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,post_away_score,post_home_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bauer_units,is_strike,pitch_count,swing_miss,plate_-x,pfx_-x
0,FF,2021-10-03,92.3,1.40,6.80,"Smith, Will",596019,519293,out,hit_into_play,1.0,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,X,5.0,ground_ball,1,2,2021,8.28,16.56,-0.69,2.83,NaN,NaN,NaN,2,9,Top,77.54,155.31,-6.833043,-134.166485,-7.361843,9.708393,26.562803,-14.083224,3.32,1.51,13.0,95.2,-13.0,92.8,2330.0,6.5,632254,54.03,0.174,0.158,0.0,1.0,0.0,0.0,2.0,61,4,4-Seam Fastball,5,0,0,5,Infield shift,Strategic,148.0,0.0,-0.073,25.243770,1,1-2,0,0.69,-8.28
1,SL,2021-10-03,80.6,1.60,6.64,"Smith, Will",596019,519293,NaN,foul,4.0,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,S,NaN,NaN,1,1,2021,-9.24,5.76,-0.71,2.62,NaN,NaN,NaN,2,9,Top,NaN,NaN,-3.700232,-117.430885,-3.266842,-6.531123,19.793390,-27.369114,3.32,1.51,108.0,75.3,75.0,81.2,2254.0,6.4,632254,54.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,3,Slider,5,0,0,5,Infield shift,Strategic,315.0,0.0,-0.027,27.965261,1,1-1,0,0.71,9.24
2,CU,2021-10-03,75.5,1.46,6.88,"Smith, Will",596019,519293,NaN,foul,5.0,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,S,NaN,NaN,1,0,2021,-7.80,-6.12,-0.04,2.46,NaN,NaN,NaN,2,9,Top,NaN,NaN,-1.977183,-109.901781,-1.155694,-4.872924,20.602334,-36.262184,3.32,1.51,157.0,83.5,65.0,75.2,1940.0,6.2,632254,54.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,2,Curveball,5,0,0,5,Infield shift,Standard,328.0,0.0,-0.020,25.695364,1,1-0,0,0.04,7.80


In [3]:
pitch = data[['pitch_type', 'player_name','release_speed', 
                   'release_spin_rate', 'spin_axis', 'pfx_-x', 
                   'pfx_z','release_extension', 'release_pos_z', 
                   'plate_-x', 'plate_x', 'plate_z', 'zone', 'pitch_name']].copy()

pitch.dropna(inplace = True)
print(pitch.shape)
pitch.head()

(705445, 14)


,pitch_type,player_name,release_speed,release_spin_rate,spin_axis,pfx_-x,pfx_z,release_extension,release_pos_z,plate_-x,plate_x,plate_z,zone,pitch_name
0,FF,"Smith, Will",92.3,2330.0,148.0,-8.28,16.56,6.5,6.80,0.69,-0.69,2.83,1.0,4-Seam Fastball
1,SL,"Smith, Will",80.6,2254.0,315.0,9.24,5.76,6.4,6.64,0.71,-0.71,2.62,4.0,Slider
2,CU,"Smith, Will",75.5,1940.0,328.0,7.80,-6.12,6.2,6.88,0.04,-0.04,2.46,5.0,Curveball
3,CU,"Smith, Will",75.0,2017.0,330.0,8.28,-8.28,5.9,6.83,-2.10,2.10,3.89,12.0,Curveball
4,FF,"Smith, Will",91.2,2281.0,143.0,-7.56,15.36,6.3,6.66,0.31,-0.31,2.80,4.0,4-Seam Fastball


In [4]:
pitch.groupby(['pitch_name']).mean()

,release_speed,release_spin_rate,spin_axis,pfx_-x,pfx_z,release_extension,release_pos_z,plate_-x,plate_x,plate_z,zone
pitch_name,,,,,,,,,,,
4-Seam Fastball,93.700994,2273.330143,192.843467,2.841528,16.101758,6.367582,5.916418,-0.001913,0.001913,2.743496,8.034226
Changeup,84.807142,1749.696269,195.017739,3.596463,6.829364,6.306857,5.796725,0.040616,-0.040616,1.850552,10.337616
Curveball,78.743635,2507.162103,124.762536,-3.862587,-8.749849,6.207957,5.974373,-0.068278,0.068278,1.833569,9.950486
Cutter,88.527884,2371.345491,174.076764,-1.365751,7.946024,6.258936,5.897058,-0.158996,0.158996,2.330472,9.014106
Knuckle Curve,81.065056,2536.118821,77.332553,-6.285540,-11.293460,6.265553,5.978711,-0.104382,0.104382,1.795782,10.018865
Sinker,93.006576,2123.460728,194.738129,5.647138,9.230332,6.307078,5.730818,0.076384,-0.076384,2.335285,8.741720
Slider,84.665382,2416.428359,145.967975,-3.219395,1.721364,6.258914,5.798154,-0.232157,0.232157,1.886006,10.004376
Split-Finger,85.705460,1364.779008,227.122671,9.603752,3.877842,6.281286,5.934464,0.222921,-0.222921,1.686196,10.548607
